In [113]:
# this notebook focuses on streamwise grain abrasion, 
# but does not account for fraction of bed exposed
# or variable streamwise lithology (ie, variable streamwise contribution from plucking)

In [114]:
import numpy as np
import matplotlib.pyplot as plt

In [124]:
# grid setup and elevation stuff
dx = 1
x = np.arange(0, 100 + dx, dx)
slope = -0.2
elev = (slope * x) + 2000
baselevel_rate = 0.01
uplift_rate = 0.001

# initial lithology distribution
percent_ig = 0.70
percent_mtm = 0.05
percent_sed = 0.25

# sed flux
qs_ig = np.zeros(len(x))
qs_mtm = np.zeros(len(x))
qs_sed = np.zeros(len(x))
qs_ig[0] = 1
qs_mtm[0] = 1
qs_sed[0] = 1

# discharge
q = x

# c is fxn of grainsize, among other stuff. Lower for igneous rocks because they comprise the gravel fraction?
# or should c be the same for all?
c_ig = 0.001
c_mtm = 0.01
c_sed = 0.05
c = 1

# erodibility
k = 0.001

# abrasion coefficients of different lithologies
beta_ig = 10
beta_mtm = 100
beta_sed = 500

# timestep stuff
num_steps = 10000
dt = 0.2 * dx * dx / c_mtm  # time-step size

In [125]:
for i in range(100):
    
    # adjust elevations and calc slope
    elev += uplift_rate * dt
    S = np.diff(elev)/dx
    
    # erode the bed
    eroded_matl = k * q[1:] * S
    
    # calc sed flux
    ig_attrition = eroded_matl * percent_ig * beta_ig
    qs_ig[1:] = (q[1:] * S * c) - ig_attrition
    # igneous += qs_ig
    
    mtm_attrition = eroded_matl * percent_mtm * beta_mtm
    qs_mtm[1:] = (q[1:] * S * c) - mtm_attrition
    # metamorphic += qs_mtm
    
    sed_attrition = eroded_matl * percent_sed * beta_sed
    qs_sed[1:] = (q[1:] * S * c) - sed_attrition
    # sedimentary += qs_sed
    
    qs_total = qs_ig + qs_mtm + qs_sed
    
    # calc change through time
    elev[:-1] -= eroded_matl - qs_total[:-1]

In [126]:
total_qs = np.sum(qs_total)
total_ig = np.sum(qs_ig)
total_mtm = np.sum(qs_mtm)
total_sed = np.sum(qs_sed)

print((total_ig/total_qs)*100)
print((total_mtm/total_qs)*100)
print((total_sed/total_qs)*100)

34.68389800908164
34.75375480265461
30.562347188265136
